# Anaconda環境下で動作可能です

In [5]:
import mne # mneライブラリ
import numpy as np # numpy: 数値計算ライブラリ
# print(__doc__)
import matplotlib # matplotlib: 図表示ライブラリ
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
from mne import io, Epochs, read_epochs
from scipy.spatial.distance import cdist, squareform
import matplotlib.font_manager as fm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import mne_rsa

ModuleNotFoundError: No module named 'mne_rsa'

In [ ]:
# 読み込んだデータの"info"情報の確認
raw.info

In [ ]:
# 読み込んだデータの測定値の読み込み
data = raw.get_data()

In [ ]:
type(data)

In [ ]:
# データ構造を確認
data.shape

In [ ]:
data

# 正しい解析のため、測定データの中身を修正する

In [ ]:
raw.info

In [ ]:
# 測定値の校正係数の二重掛けを補正する
range_factor = 100.0 / 4.5
raw_length = len(raw)
channel_length = len(raw.info["chs"])
raw_array = np.zeros((channel_length, raw_length))
for i in range(channel_length):
    cal_factor = raw.info["chs"][i]["cal"]
    if i < 64:
        raw_array[i] = (data[i] / cal_factor) * range_factor
    else:
        raw_array[i] = data[i]
raw_new = mne.io.RawArray(raw_array, raw.info)

In [ ]:
raw_new.info["chs"]

In [ ]:
# コイル番号を解析可能なコイル番号に上書きする。
for ch in raw_new.info["chs"]:
    ch["coil_type"] = 5001

In [ ]:
# 上書きされたことを確認
raw_new.info["chs"][0]["coil_type"]

In [ ]:
# 修正したデータを新たにFIFF形式で出力する
raw_new.save(modified_path, overwrite=True)

# 上で作成した修正した測定データ読み込み
## data_pathは読み込みたいデータのディレクトリに合わせて編集ください。

In [11]:
data_path = modified_path
raw = mne.io.read_raw_fif(data_path, preload=True)

Opening raw data file /Users/jiontominaga/ThesisProject/data/5014/MEG_5014/5014_02_modified_raw.fif...
Isotrak not found
    Range : 0 ... 1364499 =      0.000 ...   545.800 secs
Ready.
Reading 0 ... 1364499  =      0.000 ...   545.800 secs...


In [12]:
# データをプロットすることで確認
# raw.plot()

# STIMチャンネルにノイズがあり、そのノイズ除去を行う

In [13]:
data = raw.get_data()

In [14]:
# ノイズとする閾値を設定するため、そのオーダーを確認する
time_start = 0.0
time_stop = 700
# time_stop = 768.0
start, stop = raw.time_as_index([time_start, time_stop])
# plt.plot(np.arange(start, stop)/1000.0, data[64][start:stop])
# plt.plot(np.arange(start, stop)/1000.0, data[65][start:stop])
# plt.plot(np.arange(start, stop)/1000.0, data[66][start:stop])
# plt.plot(np.arange(start, stop)/1000.0, data[67][start:stop])

In [15]:
# ノイズとする閾値を1.0に設定し、STIM1～4のノイズを除去する
for cnt in [64, 65, 66, 67]:
    time_index = np.where(data[cnt] < 1.0)
    data[cnt][time_index] = 0.0

In [16]:
# ノイズ除去したデータで新たにデータフレーム作成
raw = mne.io.RawArray(data, raw.info)

Creating RawArray with float64 data, n_channels=68, n_times=1364500
    Range : 0 ... 1364499 =      0.000 ...   545.800 secs
Ready.


In [17]:
# データをプロットすることで確認
# raw.plot()

# STIMのノイズ除去したデータで解析を行う

In [18]:
raw.info

<Info | 10 non-empty values
 acq_pars: SHI MEG
 bads: []
 ch_names: C03, L04, L10, L02, L18, C02, L34, L26, L28, L42, L08, L48, L22, ...
 chs: 64 Gradiometers, 4 Stimulus
 custom_ref_applied: False
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 lowpass: 1132.5 Hz
 meas_date: 1970-01-01 00:00:00 UTC
 meas_id: 4 items (dict)
 nchan: 68
 projs: []
 sfreq: 2500.0 Hz
>

In [19]:
print(raw.info['highpass'])
print(raw.info['lowpass'])

0.10000000149011612
1132.5


In [20]:
# 全64チャンネルの測定結果のノイズ除去
raw.filter(l_freq=0.1, h_freq=40.0)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 82501 samples (33.000 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    3.7s finished


<RawArray | 68 x 1364500 (545.8 s), ~708.0 MB, data loaded>

In [22]:
# STIM1～3の出力からイベントを定義する
# (min_duration=0.002 and min_duration=0.3 won't have difference on the number of events detected)
events_stim1 = mne.find_events(raw, stim_channel='STIM1', min_duration=0.3)
events_stim2 = mne.find_events(raw, stim_channel='STIM2', min_duration=0.3)
events_stim3 = mne.find_events(raw, stim_channel='STIM3', min_duration=0.3)
events_stim4 = mne.find_events(raw, stim_channel='STIM1', min_duration=0.6)

print(f'stim1: {events_stim1.shape}')
print(f'stim2: {events_stim2.shape}')
print(f'stim3: {events_stim3.shape}')
print(f'stim4: {events_stim4.shape}')

#mne.find_events returns arrays of integers with the following structure: [event time in samples, this column can be ignored, event id]

Removing orphaned offset at the beginning of the file.
48 events found
Event IDs: [4]
45 events found
Event IDs: [4]
Removing orphaned offset at the beginning of the file.
39 events found
Event IDs: [4]
Removing orphaned offset at the beginning of the file.
8 events found
Event IDs: [4]
stim1: (48, 3)
stim2: (45, 3)
stim3: (39, 3)
stim4: (8, 3)
Removing orphaned offset at the beginning of the file.
48 events found
Event IDs: [4]
45 events found
Event IDs: [4]
Removing orphaned offset at the beginning of the file.
39 events found
Event IDs: [4]
Removing orphaned offset at the beginning of the file.
8 events found
Event IDs: [4]
stim1: (48, 3)
stim2: (45, 3)
stim3: (39, 3)
stim4: (8, 3)


In [23]:
#Animal (1): STIM1 (events_stim1)
events_stim_index = []
for value in events_stim1[:,0]:
    a = np.where(events_stim2[:,0]==value)[0]
    if a.size == 1:
        events_stim_index.append(a[0]) #stim(1)(1,2)(1,3)(1,2,3) n stim(1,2)(2)(2,3)(1,2,3) = stim(1,2)(1,2,3)
events_temp = np.delete(events_stim1, np.array(events_stim_index), 0) #stim(1)(1,2)(1,3)(1,2,3) - stim(1,2)(1,2,3) = stim(1)(1,3)
events_stim_index = []
for value in events_stim3[:,0]:
    a = np.where(events_temp[:,0]==value)[0] #stim(1)(1,3) n stim(1,3)(2,3)(3)(1,2,3) = stim(1,3)
    if a.size == 1:
        events_stim_index.append(a[0]) # [stim(1,3)]
events_condition1 = np.delete(events_temp, np.array(events_stim_index), 0) # stim(1)(1,3) - stim(1,3) = stim(1)
print(events_condition1.shape)

(18, 3)


In [24]:
#Human (2): STIM2 (events_stim2)
events_stim_index = []
for value in events_stim1[:,0]:
    a = np.where(events_stim2[:,0]==value)[0]
    if a.size == 1:
        events_stim_index.append(a[0]) #stim(1)(1,2)(1,3)(1,2,3) n stim(1,2)(2)(2,3)(1,2,3) = stim(1,2)(1,2,3)
events_temp = np.delete(events_stim2, np.array(events_stim_index), 0) #stim(1,2)(2)(2,3)(1,2,3) - stim(1,2)(1,2,3) = stim(2)(2,3)
events_stim_index = []
for value in events_stim3[:,0]:
    a = np.where(events_temp[:,0]==value)[0] #stim(2)(2,3) n stim(1,3)(2,3)(3)(1,2,3) = stim(2,3)
    if a.size == 1:
        events_stim_index.append(a[0]) # [stim(2,3)]
events_condition2 = np.delete(events_temp, np.array(events_stim_index), 0) # stim(2)(2,3) - stim(2,3) = stim(2)
print(events_condition2.shape)

(13, 3)


In [25]:
#Body Parts (3): STIM1, STIM2 (events_stim1)(events_stim2)
intersect = np.intersect1d(events_stim1[:,0], events_stim2[:,0]) #stim(1)(1,2)(1,3)(1,2,3) n stim(1,2)(2)(2,3)(1,2,3) = stim(1,2)(1,2,3)

events_stim_index = np.zeros_like(intersect) #empty array with shape of stim(1,2)(1,2,3)

for cnt in range(len(intersect)):
    events_stim_index[cnt] = np.where(events_stim1[:,0]==intersect[cnt])[0][0] #stim(1)(1,2)(1,3)(1,2,3) n stim(1,2)(1,2,3) = stim(1,2)(1,2,3) (stores it in events_stim_index)

events_stim_temp = events_stim1[events_stim_index] #stim(1,2)(1,2,3) *get data corresponding to indices in events_stim_index

events_stim_index = []
for value in events_stim3[:,0]: #choose event_stim that hasn't been used previously.
    a = np.where(events_stim_temp[:,0]==value)[0] #stim(1,2)(1,2,3) n stim(1,3)(2,3)(3)(1,2,3) = stim(1,2,3)
    if a.size == 1:
        events_stim_index.append(a[0]) #stim(1,2,3)
events_stim_index = np.array(events_stim_index) #stim(1,2,3) (np.array)
events_condition3 = np.delete(events_stim_temp, events_stim_index, 0) #stim(1,2)(1,2,3) - stim(1,2,3) = stim(1,2)
print(events_condition3.shape)

(11, 3)


In [26]:
#Vehicle (4): STIM3 (events_stim3)
events_stim_index = []
for value in events_stim1[:,0]:
    a = np.where(events_stim3[:,0]==value)[0]
    if a.size == 1:
        events_stim_index.append(a[0]) #stim(1)(1,2)(1,3)(1,2,3) n stim(1,3)(2,3)(3)(1,2,3) = stim(1,3)(1,2,3)
events_temp = np.delete(events_stim3, np.array(events_stim_index), 0) #stim(1,3)(2,3)(3)(1,2,3) - stim(1,3)(1,2,3) = stim(2,3)(3)
events_stim_index = []
for value in events_stim2[:,0]:
    a = np.where(events_temp[:,0]==value)[0] #stim(2,3)(3) n stim(1,2)(2)(2,3)(1,2,3) = stim(2,3)
    if a.size == 1:
        events_stim_index.append(a[0]) # [stim(2,3)]
events_condition4 = np.delete(events_temp, np.array(events_stim_index), 0) # stim(2,3)(3) - stim(2,3) = stim(3)
print(events_condition4.shape)

(8, 3)


In [27]:
#Food (5): STIM1, STIM3 (events_stim1)(events_stim3)
intersect = np.intersect1d(events_stim1[:,0], events_stim3[:,0]) #stim(1)(1,2)(1,3)(1,2,3) n stim(1,3)(2,3)(3)(1,2,3) = stim(1,3)(1,2,3)

events_stim_index = np.zeros_like(intersect) #empty array with shape of stim(1,3)(1,2,3)

for cnt in range(len(intersect)):
    events_stim_index[cnt] = np.where(events_stim1[:,0]==intersect[cnt])[0][0] #stim(1)(1,2)(1,3)(1,2,3) n stim(1,3)(1,2,3) = stim(1,3)(1,2,3) (stores corresponding indices from events_stim1 to events_stim_index)

events_stim_temp = events_stim1[events_stim_index] #stim(1,3)(1,2,3) *get data corresponding to indices in events_stim_index

events_stim_index = []
for value in events_stim2[:,0]: #choose event_stim that hasn't been used previously.
    a = np.where(events_stim_temp[:,0]==value)[0] #stim(1,3)(1,2,3) n stim(1,2)(2)(2,3)(1,2,3) = stim(1,2,3)
    if a.size == 1:
        events_stim_index.append(a[0]) #stim(1,2,3)
events_stim_index = np.array(events_stim_index) #stim(1,2,3) (np.array)
events_condition5 = np.delete(events_stim_temp, events_stim_index, 0) #stim(1,3)(1,2,3) - stim(1,2,3) = stim(1,3)
print(events_condition5.shape)

(10, 3)


In [28]:
#Inanimate (6): STIM2, STIM3 (events_stim2)(events_stim3)
intersect = np.intersect1d(events_stim2[:,0], events_stim3[:,0]) #stim(1,2)(2)(2,3)(1,2,3) n stim(1,3)(2,3)(3)(1,2,3) = stim(2,3)(1,2,3)

events_stim_index = np.zeros_like(intersect) #empty array with shape of stim(2,3)(1,2,3)

for cnt in range(len(intersect)):
    events_stim_index[cnt] = np.where(events_stim2[:,0]==intersect[cnt])[0][0] #stim(1,2)(2)(2,3)(1,2,3) n stim(2,3)(1,2,3) = stim(2,3)(1,2,3) (stores corresponding indices from events_stim1 to events_stim_index)

events_stim_temp = events_stim2[events_stim_index] #stim(2,3)(1,2,3) *get data corresponding to indices in events_stim_index

events_stim_index = []
for value in events_stim1[:,0]: #choose event_stim that hasn't been used previously.
    a = np.where(events_stim_temp[:,0]==value)[0] #stim(2,3)(1,2,3) n stim(1)(1,2)(1,3)(1,2,3) = stim(1,2,3)
    if a.size == 1:
        events_stim_index.append(a[0]) #stim(1,2,3)
events_stim_index = np.array(events_stim_index) #stim(1,2,3) (np.array)
events_condition6 = np.delete(events_stim_temp, events_stim_index, 0) #stim(2,3)(1,2,3) - stim(1,2,3) = stim(2,3)
print(events_condition6.shape)

(13, 3)


In [29]:
#Man-made Place (7): STIM1, STIM2, STIM3 (events_stim1)(events_stim2)(events_stim3)

intersect = np.intersect1d(events_stim1[:,0], events_stim2[:,0]) #stim(1)(1,2)(1,3)(1,2,3) n stim(1,2)(2)(2,3)(1,2,3) = stim(1,2)(1,2,3)
intersect2 = np.intersect1d(intersect, events_stim3[:,0]) #stim(1,2)(1,2,3) n stim(1,3)(2,3)(3)(1,2,3) = stim(1,2,3)

events_stim_index2 = np.zeros_like(intersect2) #empty array of size stim(1,2,3)

for cnt in range(len(intersect2)):
    events_stim_index2[cnt] = np.where(events_stim1[:,0]==intersect2[cnt])[0][0] #get indices of stim(1,2,3) within stim1

events_stim_temp2 = events_stim1[events_stim_index2] #get data of stim(1,2,3) from stim1

events_condition7 = events_stim_temp2
print(events_condition7.shape)

(8, 3)


In [30]:
# #Man-made Place***** (7): STIM1, STIM2, STIM3 (events_stim1)(events_stim2)(events_stim3)
#
# intersect = np.intersect1d(events_stim1[:,0], events_stim2[:,0]) #stim(1)(1,2)(1,3)(1,2,3) n stim(1,2)(2)(2,3)(1,2,3) = stim(1,2)(1,2,3)
# intersect2 = np.intersect1d(events_stim1[:,0], events_stim3[:,0]) #stim(1)(1,2)(1,3)(1,2,3) n stim(1,3)(2,3)(3)(1,2,3) = stim(1,3)(1,2,3)
# intersect3 = np.intersect1d(intersect, intersect2)
#
# events_stim_index = np.zeros_like(intersect3)
#
# for cnt in range(len(intersect3)):
#     events_stim_index[cnt] = np.where(events_stim1[:,0]==intersect3[cnt])[0][0] #get indices of stim(1,2,3) within stim1
#
# events_stim_temp = events_stim1[events_stim_index] #get data of stim(1,2,3) from stim1
#
# events_condition7 = events_stim_temp
# print(events_condition7.shape)

In [31]:
#Tools/Artifacts (8): STIM1* (events_stim4)
events_condition8 = events_stim4
print(events_condition8.shape)

(8, 3)


In [32]:
len(events_condition1),len(events_condition2),len(events_condition3), len(events_condition4),len(events_condition5),len(events_condition6), len(events_condition7),len(events_condition8)

(18, 13, 11, 8, 10, 13, 8, 8)

In [33]:
# この後の解析で、測定データから除く数値域を定義する
reject = dict(grad = 7000e-13) # T / m (gradiometers)
# reject = None

In [34]:
# イベント行列からエポックの切り出しを行う
epochs_condition1 = mne.Epochs(raw=raw, events=events_condition1, tmin=epoch_tmin, tmax=epoch_tmax, preload=True,\
                    baseline=(epoch_base,0), reject=reject)
epochs_condition2 = mne.Epochs(raw=raw, events=events_condition2, tmin=epoch_tmin, tmax=epoch_tmax, preload=True,\
                    baseline=(epoch_base,0), reject=reject)
epochs_condition3 = mne.Epochs(raw=raw, events=events_condition3, tmin=epoch_tmin, tmax=epoch_tmax, preload=True,\
                    baseline=(epoch_base,0), reject=reject)
epochs_condition4 = mne.Epochs(raw=raw, events=events_condition4, tmin=epoch_tmin, tmax=epoch_tmax, preload=True,\
                    baseline=(epoch_base,0), reject=reject)
epochs_condition5 = mne.Epochs(raw=raw, events=events_condition5, tmin=epoch_tmin, tmax=epoch_tmax, preload=True,\
                    baseline=(epoch_base,0), reject=reject)
epochs_condition6 = mne.Epochs(raw=raw, events=events_condition6, tmin=epoch_tmin, tmax=epoch_tmax, preload=True,\
                    baseline=(epoch_base,0), reject=reject)
epochs_condition7 = mne.Epochs(raw=raw, events=events_condition7, tmin=epoch_tmin, tmax=epoch_tmax, preload=True,\
                    baseline=(epoch_base,0), reject=reject)
epochs_condition8 = mne.Epochs(raw=raw, events=events_condition8, tmin=epoch_tmin, tmax=epoch_tmax, preload=True,\
                    baseline=(epoch_base,0), reject=reject)
epochs_list = (len(epochs_condition1), len(epochs_condition2), len(epochs_condition3), len(epochs_condition4), len(epochs_condition5), len(epochs_condition6), len(epochs_condition7), len(epochs_condition8), )

print(epochs_list)

Not setting metadata
18 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 18 events and 5001 original time points ...
0 bad epochs dropped
Not setting metadata
13 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 13 events and 5001 original time points ...
0 bad epochs dropped
Not setting metadata
11 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 11 events and 5001 original time points ...
0 bad epochs dropped
Not setting metadata
8 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8 events and 5001 original time points ...
0 bad epochs dropped
Not setting metadata
10 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from prelo

In [ ]:
# #Butterfly plot for Condition1
# evoked_condition1 = epochs_condition1.average()
# drop_channels = ['C03', 'L48', 'L12', 'L20']
# evoked_condition1.drop_channels(drop_channels)
#
# # バラフライプロットをするためのパラメータ定義
# # ts_args = dict(gfp=False, time_unit='ms', ylim=dict(grad=[-70, 60]))
# ts_args = dict(gfp=False, time_unit='ms')
# topomap_args = dict(sensors=True, time_unit='ms')
#
# # バタフライプロットとトポマップを表示
# title=f'Butterfly_Raw_{exp_type}_Cond1'
# fig = evoked_condition1.plot_joint(times='peaks', title=title, ts_args=ts_args, topomap_args=topomap_args)
# fig.savefig(f'{title}.png', dpi=300)
# plt.close()

In [ ]:
# #Butterfly plot for Condition2
# evoked_condition2 = epochs_condition2.average()
# drop_channels = ['C03', 'L48', 'L12', 'L20']
# evoked_condition2.drop_channels(drop_channels)
#
# # バラフライプロットをするためのパラメータ定義
# # ts_args = dict(gfp=False, time_unit='ms', ylim=dict(grad=[-70, 60]))
# ts_args = dict(gfp=False, time_unit='ms')
# topomap_args = dict(sensors=True, time_unit='ms')
#
# # バタフライプロットとトポマップを表示
# title=f'Butterfly_Raw_{exp_type}_Cond2'
# fig = evoked_condition2.plot_joint(times=[-0.5, 1.5], title=title, ts_args=ts_args, topomap_args=topomap_args)
# fig.savefig(f'{title}.png', dpi=300)
# plt.close()

In [ ]:
# #Butterfly plot for Condition3
# evoked_condition3 = epochs_condition3.average()
# drop_channels = ['C03', 'L48', 'L12', 'L20']
# evoked_condition3.drop_channels(drop_channels)
#
# # バラフライプロットをするためのパラメータ定義
# # ts_args = dict(gfp=False, time_unit='ms', ylim=dict(grad=[-70, 60]))
# ts_args = dict(gfp=False, time_unit='ms')
# topomap_args = dict(sensors=True, time_unit='ms')
#
# # バタフライプロットとトポマップを表示
# title=f'Butterfly_Raw_{exp_type}_Cond3'
# fig = evoked_condition3.plot_joint(times=[-0.5, 1.5], title=title, ts_args=ts_args, topomap_args=topomap_args)
# fig.savefig(f'{title}.png', dpi=300)
# plt.close()

In [ ]:
# #Butterfly plot for Condition4
# evoked_condition4 = epochs_condition4.average()
# drop_channels = ['C03', 'L48', 'L12', 'L20']
# evoked_condition4.drop_channels(drop_channels)
#
# # バラフライプロットをするためのパラメータ定義
# # ts_args = dict(gfp=False, time_unit='ms', ylim=dict(grad=[-70, 60]))
# ts_args = dict(gfp=False, time_unit='ms')
# topomap_args = dict(sensors=True, time_unit='ms')
#
# # バタフライプロットとトポマップを表示
# title=f'Butterfly_Raw_{exp_type}_Cond4'
# fig = evoked_condition4.plot_joint(times=[-0.5, 1.5], title=title, ts_args=ts_args, topomap_args=topomap_args)
# fig.savefig(f'{title}.png', dpi=300)
# plt.close()

In [ ]:
# #Butterfly plot for Condition5
# evoked_condition5 = epochs_condition5.average()
# drop_channels = ['C03', 'L48', 'L12', 'L20']
# evoked_condition5.drop_channels(drop_channels)
#
# # バラフライプロットをするためのパラメータ定義
# # ts_args = dict(gfp=False, time_unit='ms', ylim=dict(grad=[-70, 60]))
# ts_args = dict(gfp=False, time_unit='ms')
# topomap_args = dict(sensors=True, time_unit='ms')
#
# # バタフライプロットとトポマップを表示
# title=f'Butterfly_Raw_{exp_type}_Cond5'
# fig = evoked_condition5.plot_joint(times=[-0.5, 1.5], title=title, ts_args=ts_args, topomap_args=topomap_args)
# fig.savefig(f'{title}.png', dpi=300)
# plt.close()

In [ ]:
# #Butterfly plot for Condition6
# evoked_condition6 = epochs_condition6.average()
# drop_channels = ['C03', 'L48', 'L12', 'L20']
# evoked_condition6.drop_channels(drop_channels)
#
# # バラフライプロットをするためのパラメータ定義
# # ts_args = dict(gfp=False, time_unit='ms', ylim=dict(grad=[-70, 60]))
# ts_args = dict(gfp=False, time_unit='ms')
# topomap_args = dict(sensors=True, time_unit='ms')
#
# # バタフライプロットとトポマップを表示
# title=f'Butterfly_Raw_{exp_type}_Cond6'
# fig = evoked_condition6.plot_joint(times=[-0.5, 1.5], title=title, ts_args=ts_args, topomap_args=topomap_args)
# fig.savefig(f'{title}.png', dpi=300)
# plt.close()

In [ ]:
# #Butterfly plot for Condition7
# evoked_condition7 = epochs_condition7.average()
# drop_channels = ['C03', 'L48', 'L12', 'L20']
# evoked_condition7.drop_channels(drop_channels)
#
# # バラフライプロットをするためのパラメータ定義
# # ts_args = dict(gfp=False, time_unit='ms', ylim=dict(grad=[-70, 60]))
# ts_args = dict(gfp=False, time_unit='ms')
# topomap_args = dict(sensors=True, time_unit='ms')
#
# # バタフライプロットとトポマップを表示
# title=f'Butterfly_Raw_{exp_type}_Cond7'
# fig = evoked_condition7.plot_joint(times=[-0.5, 1.5], title=title, ts_args=ts_args, topomap_args=topomap_args)
# fig.savefig(f'{title}.png', dpi=300)
# plt.close()

In [ ]:
# #Butterfly plot for Condition8
# evoked_condition8 = epochs_condition8.average()
# drop_channels = ['C03', 'L48', 'L12', 'L20']
# evoked_condition8.drop_channels(drop_channels)
#
# # バラフライプロットをするためのパラメータ定義
# # ts_args = dict(gfp=False, time_unit='ms', ylim=dict(grad=[-70, 60]))
# ts_args = dict(gfp=False, time_unit='ms')
# topomap_args = dict(sensors=True, time_unit='ms')
#
# # バタフライプロットとトポマップを表示
# title=f'Butterfly_Raw_{exp_type}_Cond8'
# fig = evoked_condition8.plot_joint(times=[-0.5, 1.5], title=title, ts_args=ts_args, topomap_args=topomap_args)
# fig.savefig(f'{title}.png', dpi=300)
# plt.close()

In [35]:
epochs_condition1.drop_channels(ch_names=['STIM1', 'STIM2', 'STIM3', 'STIM4'])
epochs_condition1.drop_channels(ch_names=['C03', 'L48', 'L32', 'L12', 'L20'])
print(f'Number of MEG channels for epochs_condition1: {len(epochs_condition1.ch_names)}')

epochs_condition2.drop_channels(ch_names=['STIM1', 'STIM2', 'STIM3', 'STIM4'])
epochs_condition2.drop_channels(ch_names=['C03', 'L48', 'L32', 'L12', 'L20'])
print(f'Number of MEG channels for epochs_condition2: {len(epochs_condition2.ch_names)}')

epochs_condition3.drop_channels(ch_names=['STIM1', 'STIM2', 'STIM3', 'STIM4'])
epochs_condition3.drop_channels(ch_names=['C03', 'L48', 'L32', 'L12', 'L20'])
print(f'Number of MEG channels for epochs_condition3: {len(epochs_condition3.ch_names)}')

epochs_condition4.drop_channels(ch_names=['STIM1', 'STIM2', 'STIM3', 'STIM4'])
epochs_condition4.drop_channels(ch_names=['C03', 'L48', 'L32', 'L12', 'L20'])
print(f'Number of MEG channels for epochs_condition4: {len(epochs_condition4.ch_names)}')

epochs_condition5.drop_channels(ch_names=['STIM1', 'STIM2', 'STIM3', 'STIM4'])
epochs_condition5.drop_channels(ch_names=['C03', 'L48', 'L32', 'L12', 'L20'])
print(f'Number of MEG channels for epochs_condition5: {len(epochs_condition5.ch_names)}')

epochs_condition6.drop_channels(ch_names=['STIM1', 'STIM2', 'STIM3', 'STIM4'])
epochs_condition6.drop_channels(ch_names=['C03', 'L48', 'L32', 'L12', 'L20'])
print(f'Number of MEG channels for epochs_condition6: {len(epochs_condition6.ch_names)}')

epochs_condition7.drop_channels(ch_names=['STIM1', 'STIM2', 'STIM3', 'STIM4'])
epochs_condition7.drop_channels(ch_names=['C03', 'L48', 'L32', 'L12', 'L20'])
print(f'Number of MEG channels for epochs_condition7: {len(epochs_condition7.ch_names)}')

epochs_condition8.drop_channels(ch_names=['STIM1', 'STIM2', 'STIM3', 'STIM4'])
epochs_condition8.drop_channels(ch_names=['C03', 'L48', 'L32', 'L12', 'L20'])
print(f'Number of MEG channels for epochs_condition8: {len(epochs_condition8.ch_names)}')

Number of MEG channels for epochs_condition1: 59
Number of MEG channels for epochs_condition2: 59
Number of MEG channels for epochs_condition3: 59
Number of MEG channels for epochs_condition4: 59
Number of MEG channels for epochs_condition5: 59
Number of MEG channels for epochs_condition6: 59
Number of MEG channels for epochs_condition7: 59
Number of MEG channels for epochs_condition8: 59


In [36]:
data_cond1 = epochs_condition1.get_data()
print(data_cond1.shape)
n_epochs, n_channels, n_times = data_cond1.shape

for i in range(n_times):


(18, 59, 5001)


In [2]:
import numpy as np
x = np.arange(45).reshape(5,3,3)
print(x)
print(x.shape)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]

 [[27 28 29]
  [30 31 32]
  [33 34 35]]

 [[36 37 38]
  [39 40 41]
  [42 43 44]]]
(5, 3, 3)


In [5]:
print(x[0,:,:])

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [ ]:
#Plot bar plots for the averaged epoch spearman's correlation for each channel
condition_list = [epochs_condition1, epochs_condition2, epochs_condition3, epochs_condition4, epochs_condition5, epochs_condition6, epochs_condition7, epochs_condition8]

for condition in condition_list:
    if condition == epochs_condition1:
        condition_name = "Animal"
    elif condition == epochs_condition2:
        condition_name = "Human"
    elif condition == epochs_condition3:
        condition_name = "Body Parts"
    elif condition == epochs_condition4:
        condition_name = "Vehicle"
    elif condition == epochs_condition5:
        condition_name = "Food"
    elif condition == epochs_condition6:
        condition_name = "Inanimate"
    elif condition == epochs_condition7:
        condition_name = "Man-made Place"
    elif condition == epochs_condition8:
        condition_name = "Tools_Artifacts"

    print(f'{condition_name} shape: {condition.get_data().shape}')

    import numpy as np
    from itertools import combinations
    from scipy.stats import spearmanr
    import matplotlib.pyplot as plt

    # Load epoch data
    arr = condition.get_data()

    # Get the number of rows
    num_rows = arr.shape[0]
    num_channels = arr.shape[1]

    # Generate all combinations of epoch indices
    row_combinations = list(combinations(range(num_rows), 2))
    num_combinations = len(row_combinations)

    # Initialize the array to store the Spearman correlation coefficients
    spearman_channels = np.zeros((num_channels, num_combinations))

    # Now you can iterate over the combinations and access the corresponding rows
    for ch in range(num_channels):
        for i, combo in enumerate(row_combinations):
            epoch1, epoch2 = combo
            epoch1_data = arr[epoch1][ch]
            epoch2_data = arr[epoch2][ch]
            corr_coeff, _ = spearmanr(epoch1_data, epoch2_data)
            spearman_channels[ch,i] =corr_coeff

    print(f'Number of spearmans correlation calculated: {spearman_channels.shape}')
    print(type(spearman_channels))
    average_spearmans = np.mean(spearman_channels, axis=1)
    plt.figure().set_figwidth(15)
    plt.bar(condition.ch_names, average_spearmans)
    plt.ylabel("Spearman's Rho")
    plt.xlabel("Channel")
    plt.xticks(rotation=90)
    plt.title("("+ exp_type + ") " + "Within-category spearman's correlation" + " (" + condition_name + ")")

    home_dir = "/Users/jiontominaga/ThesisProject/"
    fig_save_dir = home_dir + subject_number + exp_type + "/"
    fig_file_name = "("+ exp_type + ") " + "Within-category spearman's correlation" + " (" + condition_name + ")"+".png"
    fig_file = fig_save_dir + fig_file_name
    plt.savefig(fig_file, dpi=300)
    plt.close()

In [ ]:
# #Plot TOPOMAP for the averaged epoch spearman's correlation for each channel
# condition_list = [epochs_condition1, epochs_condition2, epochs_condition3, epochs_condition4, epochs_condition5, epochs_condition6, epochs_condition7, epochs_condition8]
#
# for condition in condition_list:
#     if condition == epochs_condition1:
#         condition_name = "Animal"
#     elif condition == epochs_condition2:
#         condition_name = "Human"
#     elif condition == epochs_condition3:
#         condition_name = "Body Parts"
#     elif condition == epochs_condition4:
#         condition_name = "Vehicle"
#     elif condition == epochs_condition5:
#         condition_name = "Food"
#     elif condition == epochs_condition6:
#         condition_name = "Inanimate"
#     elif condition == epochs_condition7:
#         condition_name = "Man-made Place"
#     elif condition == epochs_condition8:
#         condition_name = "Tools_Artifacts"
#
#     print(f'{condition_name} shape: {condition.get_data().shape}')
#
#     import numpy as np
#     from itertools import combinations
#     from scipy.stats import spearmanr
#     import matplotlib.pyplot as plt
#
#     # Load epoch data
#     arr = condition.get_data()
#
#     # Get the number of rows
#     num_rows = arr.shape[0]
#     num_channels = arr.shape[1]
#
#     # Generate all combinations of epoch indices
#     row_combinations = list(combinations(range(num_rows), 2))
#     num_combinations = len(row_combinations)
#
#     # Initialize the array to store the Spearman correlation coefficients
#     spearman_channels = np.zeros((num_channels, num_combinations))
#
#     # Now you can iterate over the combinations and access the corresponding rows
#     for ch in range(num_channels):
#         for i, combo in enumerate(row_combinations):
#             epoch1, epoch2 = combo
#             epoch1_data = arr[epoch1][ch]
#             epoch2_data = arr[epoch2][ch]
#             corr_coeff, _ = spearmanr(epoch1_data, epoch2_data)
#             spearman_channels[ch,i] =corr_coeff
#
#     print(f'Number of spearmans correlation calculated: {spearman_channels.shape}')
#     print(type(spearman_channels))
#     average_spearmans = np.mean(spearman_channels, axis=1)
#     # mne.viz.plot_topomap(data=average_spearmans, )
#     # plt.ylabel("Spearman's Rho")
#     # plt.xlabel("Channel")
#     # plt.xticks(rotation=90)
#     # plt.title("("+ exp_type + ") " + "Within-category spearman's correlation" + " (" + condition_name + ")")
#     #
#     # home_dir = "/Users/jiontominaga/ThesisProject/"
#     # fig_save_dir = home_dir + subject_number + exp_type + "/"
#     # fig_file_name = "("+ exp_type + ") " + "Within-category spearman's correlation" + " (" + condition_name + ")"+".png"
#     # fig_file = fig_save_dir + fig_file_name
#     # plt.savefig(fig_file, dpi=300)
#     # plt.close()

In [ ]:
# ###model RDM (WordNet)###
# import numpy as np
# import sqlite3
# from itertools import product
# from nltk.corpus import wordnet
#
# def jpn_path_similarity(word1, word2, cursor):
#     cursor.execute("SELECT synset FROM sense, word WHERE sense.wordid = word.wordid AND word.lemma=?", (word1,))
#     synsets1 = cursor.fetchall()
#
#     cursor.execute("SELECT synset FROM sense, word WHERE sense.wordid = word.wordid AND word.lemma=?", (word2,))
#     synsets2 = cursor.fetchall()
#
#     max_similarity = 0
#     for synset1, synset2 in product(synsets1, synsets2):
#         synset1_id, = synset1
#         synset2_id, = synset2
#         synset1 = wordnet.synset_from_pos_and_offset(synset1_id[-1], int(synset1_id[:-2]))
#         synset2 = wordnet.synset_from_pos_and_offset(synset2_id[-1], int(synset2_id[:-2]))
#         similarity = synset1.path_similarity(synset2)
#         if similarity is not None and similarity > max_similarity:
#             max_similarity = similarity
#     return max_similarity
#
# def compute_rdm(words, cursor):
#     n = len(words)
#     rdm = np.zeros((n, n))
#     for i in range(n):
#         for j in range(i + 1, n):
#             rdm[i, j] = 1 - jpn_path_similarity(words[i], words[j], cursor)
#             rdm[j, i] = rdm[i, j]
#     return rdm
#
# excel_path = '/Users/jiontominaga/ThesisProject/data/word_list/stim_list_final3.xlsx'
# list = pd.read_excel(excel_path)
#
# word_list = list.loc[:, 'lemma']
#
# word_list = word_list.tolist()
# jpn_words = word_list
#
# # Example list of Japanese words
# # jpn_words = ['寿司', '餃子', 'クッキー', 'ナイフ', '櫛', '鉛筆',]
#
# # Connect to the Japanese WordNet SQLite database
# conn = sqlite3.connect('wnjpn.db')
# cursor = conn.cursor()
#
# # Compute the representational dissimilarity matrix
# rdm = compute_rdm(jpn_words, cursor)
# print("Representational Dissimilarity Matrix:")
# print(rdm)
#
# # Close the connection to the database
# conn.close()
#
# # Set the font that supports Japanese characters
# font_path = '/Users/jiontominaga/Library/Fonts/ipaexg.ttf'  # Update this path with the appropriate font file on your system
# font_prop = fm.FontProperties(fname=font_path)
#
# #Set the figure size
# plt.figure(figsize=(10,8))
#
# # Display the RDM using Seaborn
# sns.set_theme()
# ax = sns.heatmap(rdm, annot=False, cmap='coolwarm', square=True, linewidths=.5, cbar_kws={"shrink": .5}, xticklabels=word_list, yticklabels=word_list)
# plt.title('modelRDM(6words)_WN.png', fontproperties=font_prop)
# ax.set_xticklabels(jpn_words, fontproperties=font_prop, fontsize=12)
# ax.set_yticklabels(jpn_words, fontproperties=font_prop, fontsize=12)
# ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
#
# ax.tick_params(labelsize=5)
# plt.xticks(rotation=45)
# plt.yticks(rotation=0)
#
# # #Add category labels
# # for i, word in enumerate(japanese_words):
# #     category = word_categories[word]
# #     ax.text(
# #         i + 0.5, len(japanese_words) - 0.5, category,
# #         fontproperties=font_prop, fontsize=12, ha='center', va='center',
# #         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.2')
# #     )
#
# plt.tight_layout()
#
# # Save the plot as a high-resolution PNG image
# plt.savefig('modelRDM(80words)_WN.png', dpi=300)
#
# # Convert the RDM numpy array to a Pandas DataFrame
# rdm_df = pd.DataFrame(rdm, index=jpn_words, columns=jpn_words)
#
# # Save the RDM DataFrame as a CSV file
# rdm_df.to_csv('modelRDM(80words)_WN.csv', encoding='utf-8')
# plt.close()

In [ ]:
# ###modelRDM (Word2Vec)###
# ###Here words being loaded are limited to 50,000 words to speed up the computation###
#
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from gensim.models import KeyedVectors
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import matplotlib.font_manager as fm
#
# # Load pre-trained Japanese word embeddings
# # japanese_model = KeyedVectors.load_word2vec_format('cc.ja.300.vec', binary=False)
# japanese_model = KeyedVectors.load_word2vec_format('cc.ja.300.vec', binary=False, limit=50000)
#
# # excel_path = '/Users/jiontominaga/ThesisProject/data/word_list/stim_list_final3.xlsx'
# # list = pd.read_excel(excel_path)
# #
# # word_list = list.loc[:, 'lemma']
# #
# # word_list = word_list.tolist()
#
# word_list=['寿司', '餃子', 'クッキー', 'ナイフ', '櫛', '鉛筆',]
# print(word_list)
#
# # Input list of Japanese words
# japanese_words = word_list
#
# # #Create a mapping of words to their categories
# # word_categories_tuples = {
# #     tuple(['蟻', '牛', '象', '鯨', '蛇', '虎', 'ライオン', '蛸']):'Animals',
# #     tuple(['先生', '兵士', '医者', '赤ちゃん', 'シェフ', '警官', '娘', '息子']): 'Human',
# #     tuple(['腕', '指', '顎', '鼻', '踵', '肘', '膝', '耳']): 'Body Parts',
# #     tuple(['ボート', 'バス', '電車', 'トラック', 'バイク', 'ロケット', 'タクシー', '馬車']): 'Vehicle',
# #     tuple(['チーズ', '卵', '寿司', 'バナナ', '人参', 'トマト', 'カボチャ', 'ラーメン']): 'Food',
# #     tuple(['薔薇', '太陽', '雪', '海', '月', '雨', '桜', '向日葵']): 'Inanimate',
# #     tuple(['病院', 'ホテル', 'オフィス', '道', '橋', '駅', '神社', 'コンビニ']): 'Made-made place',
# #     tuple(['カメラ', 'フォーク', '鍵', '新聞', '鉛筆', '傘', 'ベッド', 'テレビ']) : 'Artifacts/Tools'}
# #
# # # Create a new dictionary to map individual words to their categories
# # word_categories = {}
# # for words, category in word_categories_tuples.items():
# #     for word in words:
# #         word_categories[word] = category
#
# # Convert words to their vector representations
# vectors = [japanese_model[word] for word in japanese_words]
#
# # Calculate cosine similarity matrix
# cosine_sim_matrix = cosine_similarity(vectors)
#
# # Compute RDM by subtracting the cosine similarity matrix from 1
# rdm = 1 - cosine_sim_matrix
# np.fill_diagonal(rdm, 0)
# # Set the font that supports Japanese characters
# font_path = '/Users/jiontominaga/Library/Fonts/ipaexg.ttf'  # Update this path with the appropriate font file on your system
# font_prop = fm.FontProperties(fname=font_path)
#
# #Set the figure size
# plt.figure(figsize=(10,8))
#
# # Display the RDM using Seaborn
# sns.set_theme()
# ax = sns.heatmap(rdm, annot=False, cmap='coolwarm', square=True, linewidths=.5, cbar_kws={"shrink": .5}, xticklabels=word_list, yticklabels=word_list)
# plt.title('modelRDM(6words)_w2v.png', fontproperties=font_prop)
# ax.set_xticklabels(japanese_words, fontproperties=font_prop, fontsize=12)
# ax.set_yticklabels(japanese_words, fontproperties=font_prop, fontsize=12)
# ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
#
# ax.tick_params(labelsize=5)
# plt.xticks(rotation=45)
# plt.yticks(rotation=0)
#
# # #Add category labels
# # for i, word in enumerate(japanese_words):
# #     category = word_categories[word]
# #     ax.text(
# #         i + 0.5, len(japanese_words) - 0.5, category,
# #         fontproperties=font_prop, fontsize=12, ha='center', va='center',
# #         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.2')
# #     )
#
# plt.tight_layout()
#
# # Save the plot as a high-resolution PNG image
# plt.savefig('modelRDM(6words)_w2v.png', dpi=300)
#
# # Convert the RDM numpy array to a Pandas DataFrame
# rdm_df = pd.DataFrame(rdm, index=japanese_words, columns=japanese_words)
#
# # Save the RDM DataFrame as a CSV file
# rdm_df.to_csv('modelRDM(6words)_w2v.csv', encoding='utf-8')
# plt.close()